In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd 
import numpy as np
import os 
import glob
from government_functions import * 
import ast 

In [ ]:
file_name='austria.csv'
df=process_gov_csv(file_name)

 now, onto matching :D 

In [2]:
final_df=pd.read_excel('results/complete_dataframe.xlsx',index_col=0)

In [ ]:
elections=final_df.columns.to_numpy()
austria_df=final_df[[col for col in final_df.columns if col.startswith('Austria')]] #this is case sensitive... 
austria_elections=austria_df.columns.to_numpy()

In [ ]:
test=match_ministries_and_elections('Austria',austria_elections,df)

In [ ]:
test.keys()

In [ ]:
test.get('-Sep-08')

create loop for all countries: 

In [5]:

for csv_file in glob.glob(os.path.join('government/','*.csv')): 
    file= os.path.basename(csv_file)
    countryname,_=os.path.splitext(file)
    #create country government dataframe
    df=process_gov_csv(csv_file)
    #grab part of final_df that is relevant for the country
    country_df=final_df[[col for col in final_df.columns if col.startswith(countryname)]] #this is case sensitive... 
    elections=country_df.columns.to_numpy() # list of elections in the country
    #get all election_df's (currently not really necessary but might be if stability or changes in government becomre relevant)
    election_period_dict=match_ministries_and_elections(countryname,elections,df)
    #subset election_df's for only the first ministers in an election period (initial government)
    government_dict=starting_gov_dict(election_period_dict)
    ##loop over all election periods: ##
    for date,dataframe in government_dict.items():
        if not dataframe.empty:
            party_str=country_df.at['parties',f'{countryname}{date}']
            parties=ast.literal_eval(party_str) # parses string to list  
            #create dicts
            ministry_dict,unweighted_dict,weighted_dict=get_ministry_dicts(dataframe,parties) 
            #translate dicts into lists / arrays corrosponding to the party list     
            ministy_list=[]
            unweighted_array=np.zeros(len(parties))  
            weighted_array=np.zeros(len(parties))  
            for i,(party,value) in enumerate(ministry_dict.items()): 
                ministy_list.append(value)
            for i,(party,value) in enumerate(unweighted_dict.items()):
                unweighted_array[i]=value
            for i,(party,value) in enumerate(weighted_dict.items()):
                weighted_array[i]=value
            
            # write list/arrays to final_df:
            if 'Ministers' not in final_df.index: 
                final_df.loc['Ministers']=[np.nan] * len(final_df.columns)   
            if 'unweighted' not in final_df.index: 
                final_df.loc['unweighted']=[np.nan] * len(final_df.columns)   
            if 'weighted' not in final_df.index: 
                final_df.loc['weighted']=[np.nan] * len(final_df.columns) 
            final_df.at['Ministers',f'{countryname}{date}']=ministy_list
            final_df.at['unweighted',f'{countryname}{date}']=unweighted_array
            final_df.at['weighted',f'{countryname}{date}']=weighted_array 



    



In [7]:
with pd.ExcelWriter('combined_dataframe.xlsx', engine='xlsxwriter') as writer:
    final_df.to_excel(writer)